# FastAI Movie Recommender Step-by-Step
> An app where you can type in your favorite movie and it will suggest 30 movies similar to i t using a collaborative filtering model trained on over 9million movie reviews.

In [ ]:
#| default_exp movie_recommender

### Makes sure all your libraries and packages are installed and loaded.

[Fastai](https://youtu.be/p4ZZq0736Po?t=3724) is the library we use to train, load and run our model.  NB: we already trained our model so we'll only be loading it here.

In [ ]:
#!pip install -Uqq fastai 

[Gradio](https://gradio.app/getting_started/) allows you to easily publish your model onto the web for peope to use.

In [ ]:
# !pip install gradio

[nbdev](https://nbdev.fast.ai/tutorials/tutorial.html) is what we're using to make this documentation easier and more presentable.

In [ ]:
# !pip install nbdev

In [ ]:
#| hide
from nbdev.showdoc import *

These are the libraries we need

In [ ]:
#| export
# For modeling
from fastai.tabular.all import *
from fastai.collab import *
# import torch.nn as nn
#for publishing the model
import gradio as gr

In [ ]:
# For modeling
from fastai.tabular.all import *
from fastai.collab import *
#for publishing the model
import gradio as gr

In [ ]:
#| export
path = Path('../models_and_dls')

# Load model and data loaders or movie title csv


The dataloaders we used to train the model from the 9 million reviews were 200MB so we use a csv instead. To convert from data loaders to pandas DataFrame use code hashtagged out just below

In [ ]:
# titles = pd.DataFrame(dls.classes['title']).reset_index() #create df from data loaders(dls)
# titles = titles.rename(columns={'index':'midx',0:'title'}) # rename columns

In [ ]:
#| export
#load your the model and movie titles with indexes (from the previous data loaders)
learn = load_learner(path/'movie_predictor_large20mil_639.pkl')
titles = pd.read_csv(path/'movie_20mil_dls.csv'); titles.head()

,midx,title
0,0,#na#
1,1,"""Great Performances"" Cats (1998)"
2,2,#1 Cheerleader Camp (2010)
3,3,#Horror (2015)
4,4,#realityhigh (2017)


# Functions for Model Inference

In [ ]:
#| export
def get_movie_recs(full_title:str, # String saved to `favorite_movie` variable
                   learn,# Trained Model
                   titles #DataFrame with all movie titles from dls
                        ):
    "This function will use the `nn.CosineSimilarity` on the 50 latent factors for each movie to find the 30 movies most similar to your favorite movie."
    movie_factors = learn.model.i_weight.weight
    idx = int(titles[titles.title == full_title].midx)
    distances =  nn.CosineSimilarity(dim=1)(movie_factors,movie_factors[idx][None])
    idx = distances.argsort(descending=True)
    return [o for o in titles.title[idx.tolist()][:30]]

In [ ]:
show_doc(get_movie_recs)

---

[source](https://github.com/daveramseymusic/movie-recommender/blob/main/movie_recommender/movie_recommender.py#L23){target="_blank" style="float:right; font-size:smaller"}

### get_movie_recs

>      get_movie_recs (full_title:str, learn, titles)

This function will use the `nn.CosineSimilarity` on the 50 latent factors for each movie to find the 30 movies most similar to your favorite movie.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| full_title | str | String saved to `favorite_movie` variable |
| learn |  | Trained Model |
| titles |  | DataFrame with all movie titles from dls |

In [ ]:
#| export
def search_movies_n_recommend(favorite_movie:str, # The movie title typed into the `gr.Textbox()` that the user will see on the `gradio` app
                             learn, # Trained model
                              titles, # Movie titles df
                             ):
    "This function returns 30 recommendations using `get_movie_recs` after searching through all movie titles to find any titles that contain the words in the `favorite_movie` variable."
    movies_found = ''
    s = favorite_movie.lower()
    #remove THE from the title
    if s[:4] == 'the ':
        s = s[4:]

    lst = titles['title'].tolist()
    index = []
    i=0
    length = len(lst)
    while i<length:
        if s in lst[i].lower():
            full_title = lst[i]
            movies_found+= str(full_title) +'\n'
#             print(f'Your Favorite Movies:  {full_title}')
        i+=1
    # write explaination in case multiple movies
    explainer = f'If there are multiple movies above: Please paste your favorite movie into the "favorite_movie" box.\n The box below is currently showing recommendations for the movie:  {full_title}'
    #print movies found and explainer
    output_str =  movies_found+'\n\n'+explainer
    
    #get recommendations from model
    recommendations = get_movie_recs(full_title, learn,titles) 
    #create list of all the recommendations to print
    print_lst = ''
    for o in recommendations:    print_lst =print_lst+str(o) + '\n'
    
    
#     return  f'Will Recommend for:  {full_title}', output_str, recomendations
    return output_str, f'Recommendations for {full_title}: \n\n {print_lst}'

# Gradio For Publishing the Model 
>Now we use `gr.Interface()` to create a block that contains our `search_movies_n_recommend` fuction as well as variables that dictate what will be shown once we publish it on `gradio`

In [ ]:
intf = gr.Interface(fn=search_movies_n_recommend,
                    inputs=gr.Textbox(lines=1,placeholder="Put Your Favorite Movie Here To See Similar Movie Recommendations..."),
                    outputs=["text","text"],
                    examples=['Child\'s Play (1988)','Rushmore','brothers Bloom',
                              'call me by your name', 'Jumanji (1995)','The Three Amigos'],
                    title="Dave's Fast.AI Movie Recommender",
                    article='This recommender uses a collaborative filtering model fashioned from the [Fast.AI](https://github.com/fastai/fastbook/blob/master/08_collab.ipynb) library and trained on the 9 million movie reviews from the [Movie Lense Dataset](https://grouplens.org/datasets/movielens/25m/).  It trains Latent Factors to give movies and users descriptive weights that help predict thier reivews based on these factors.  The model was trained completely WITHOUT titles, descriptions, or categories and amazingly these things can be added to imporove this model.  Enjoy!'                    )

# Actually Try the Model HERE
>Now we launch it so it can be used literally here in the `jupyter notebook`

In [ ]:
intf.launch(inline=True, #when True publishes the page in notebook
    share=False) #when True automatically publishes the app to the gradio website for 72 hours.

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App>, 'http://127.0.0.1:7861/', None)

# Example Without Gradio: 
>Here is an example of the model being used straight from the `get_movie_recs` function, recommending movies similar to one of my favorites.

In [ ]:
favorite_movie = 'Beasts of the Southern Wild (2012)'

get_movie_recs(full_title=favorite_movie, learn=learn, titles=titles)

['Beasts of the Southern Wild (2012)',
 'Mina Tannenbaum (1994)',
 'Triplets of Belleville, The (Les triplettes de Belleville) (2003)',
 'Chasing Mavericks (2012)',
 'Bye Bye Brazil (Bye Bye Brasil) (1980)',
 "Tale of Winter, A (a.k.a. A Winter's Tale) (Conte d'hiver) (1992)",
 'Coonskin (1975)',
 'Nanny, The (1965)',
 'Trials of Henry Kissinger, The (2002)',
 'Rain Fall (2009)',
 'Hail, Caesar! (2016)',
 'Freeway (1996)',
 'Phantom Boy (2015)',
 'The Attic (2008)',
 'James and the Giant Peach (1996)',
 'Song of the Sea (2014)',
 'Bitter Tears of Petra von Kant, The (bitteren Tränen der Petra von Kant, Die) (1972)',
 'The Ballad of Buster Scruggs (2018)',
 'Bank Dick, The (1940)',
 'Salvador (1986)',
 'Go-Getter, The (2007)',
 'Raining Stones (1993)',
 'Easy Street (1917)',
 'Scarlet Diva (2000)',
 'Kings of Summer, The (2013)',
 'My Side of the Mountain (1969)',
 'Nashville (1975)',
 'Fantastic Mr. Fox (2009)',
 'After the Rehearsal (Efter repetitionen) (1984)',
 'Touchy Feely (2013)'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()